In [8]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from pandas_datareader import DataReader as pdr

from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor

from modules.backtest import backtest, cumplot

In [9]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)
transform2 = QuantileTransformer(
    output_distribution="normal"
)
transform3 = QuantileTransformer(
    output_distribution="normal"
)
poly = PolynomialFeatures(degree=2)

In [10]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()

In [11]:
data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m, siccd
    from data
    where date>='2000-01'
    order by date, ticker
    """, 
    conn
)
data = data.dropna()
conn.close()

factors = pdr("F-F_Research_Data_Factors", "famafrench", start=2000)[0] / 100
factors["date"] = [str(d) for d in factors.index]
factors["market"] = factors["Mkt-RF"] + factors["RF"]
data = data.merge(factors[["date", "market"]], on="date", how="left")
data["retx"] = data.ret - data.market

In [19]:
model = TransformedTargetRegressor(
    regressor=RandomForestRegressor(random_state=0, max_depth=4),
    transformer=transform3
)

pipe = make_pipeline(
  transform1,
  poly,
  transform2,
  model
)

rets = backtest(
    data=data, 
    features=["roeq", "mom12m"], 
    target="retx", 
    pipe=pipe, 
    numstocks=100
)

with create_engine("sqlite:///files/mydata.db").connect() as conn:
    rets.to_sql("model1", conn, if_exists="replace")

with pd.ExcelWriter("files/mydata.xlsx", mode="a", if_sheet_exists="replace") as writer:
    rets.to_excel(writer, sheet_name="model1")  